In [3]:
import cv2
import os

In [4]:
def smooth_image(image_path, output_path, kernel_size=(5, 5) , output_dir='enhanced_frames'):
    """
    Apply Gaussian smoothing to an image to increase its quality.

    Parameters:
    image_path (str): Path to the input image.
    output_path (str): Path to save the smoothed image.
    kernel_size (tuple): Size of the Gaussian kernel. Default is (5, 5).
    """
    # Read the image
    image = cv2.imread(image_path)
    
    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return
    
    # Apply Gaussian smoothing
    smoothed_image = cv2.GaussianBlur(image, kernel_size, 0)

    # Create the enhanced_frames directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    
    # Save the smoothed image
    cv2.imwrite(output_path, smoothed_image)

# print(f"smoothing is done inside output_dir folder")
# Example usage
smooth_image('frames/frame_0152.png', 'enhanced_frames/frame_0152.png')

Extract frames from the video

In [5]:
# Create a directory to store the frames
if not os.path.exists('frames'):
    os.makedirs('frames')

# Open the video file
video_path = './video/film.mp4'
cap = cv2.VideoCapture(video_path)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = f'frames/frame_{frame_count:04d}.png'
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

Enhance the frames using gaussian

In [6]:
for i in range(frame_count):
    frame_filename = f'frames/frame_{i:04d}.png'
    enhanced_frame_filename = f'enhanced_frames/frame_{i:04d}.png'
    smooth_image(frame_filename, enhanced_frame_filename,kernel_size=(9,9))

Reconstruct the video from the enhanced frames

In [7]:
# Open the original video to get its properties
original_video = cv2.VideoCapture(video_path)
fps = original_video.get(cv2.CAP_PROP_FPS)
frame_width = int(original_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(original_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_video.release()

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs as well
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))
for i in range(frame_count):
    enhanced_frame_filename = f'enhanced_frames/frame_{i:04d}.png'
    enhanced_frame = cv2.imread(enhanced_frame_filename)
    
    if enhanced_frame is None:
        print(f"Error: Unable to load image at {enhanced_frame_filename}")
        continue
    
    out.write(enhanced_frame)

out.release()
cv2.destroyAllWindows()

Intialize video colrizer

In [8]:
%cd MVP/
%cd DeOldify/

[Errno 2] No such file or directory: 'MVP/'
/Users/omar/Desktop/College/GP/MVP
/Users/omar/Desktop/College/GP/MVP/DeOldify


Change this to GPU

In [9]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
# device.set(device=DeviceId.GPU0)
device.set(device=DeviceId.CPU)

<DeviceId.CPU: 99>

In [10]:
from deoldify.visualize import *
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

NameError: name 'CallbackHandler' is not defined

In [1]:
colorizer = get_video_colorizer()

NameError: name 'get_video_colorizer' is not defined

## Colorize!!

In [9]:
print(os.getcwd())

/Users/omar/Desktop/College/GP/MVP/DeOldify


In [ ]:
#NOTE:  Max is 44 with 11GB video cards.  21 is a good default
render_factor=21
#NOTE:  Make source_url None to just read from file at ./video/source/[file_name] directly without modification
# source_url='https://twitter.com/silentmoviegifs/status/1116751583386034176'
# source_url='https://www.youtube.com/watch?v=mwFbXfYhOmk'
source_url=None
file_path='../output_video.mp4'
file_name = 'output_video'
file_name_ext = file_name + '.mp4'
result_path = None

file_path =Path(file_path)

if source_url is not None:
    result_path = colorizer.colorize_from_url(source_url, file_name_ext, render_factor=render_factor)
else:
    print('Reading video from file...')
    result_path = colorizer._colorize_from_path(file_path, render_factor=render_factor)
    # result_path = colorizer.colorize_from_file_name(file_name_ext, render_factor=render_factor)

show_video_in_notebook(result_path)

Exception: Video at path specfied, video/source/output_video.mp4 could not be found.

## See how well render_factor values perform on a frame here

In [ ]:
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/' + file_name + '/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))